In [1]:
from ctxbio.imports import *

[2020-03-09 09:42:54,671] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/ssmith/benchmark/env/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/home/ssmith/benchmark/env/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
import glob

In [3]:
# adjust as appropriate for your system
basepath = "/mnt/ssd/moredata/"

### Disk Performance:

Time how long it takes to read the EBS (gp2) data:

In [14]:
!sudo bash -c 'sync; echo 1 >| /proc/sys/vm/drop_caches'

In [15]:
!time cat /mnt/ebs_gp2/moredata/*.h5ad >/dev/null


real	0m2.814s
user	0m0.012s
sys	0m0.157s


Time how long it takes to read the SSD data.

In [17]:
!sudo bash -c 'sync; echo 1 >| /proc/sys/vm/drop_caches'

In [18]:
!time cat /mnt/ssd/moredata/*.h5ad >/dev/null


real	0m1.262s
user	0m0.009s
sys	0m0.159s


#### NOTE: Though the SSD is more than 2x as fast, the process below is CPU-bound.  It will take 24s to read the files.

### Code Performance:

The next 3 sections split up the original notebook so we can see exactly which parts of the code are slow.  The original timings were:
```
CPU times: user 27 s, sys: 2.34 s, total: 29.3 s
Wall time: 29.2 s
```
The code below takes `24.5s` total, so the original code takes 20% longer.  This could be because of NFS, or because the test server has faster CPUs.

In [4]:
paths = glob.glob(os.path.join(basepath, "*.h5ad"))

In [5]:
%load_ext line_profiler

In [8]:
%%writefile load_file.py

from anndata import AnnData
import numpy as np
import tables
from ctxbio.imports import *

from scipy.sparse import csr_matrix

# Enable CUDA GPU support.
#import cupy as cp
#from cupyx.scipy.sparse import csr_matrix

def load_file(paths):
    for i, filename in enumerate(paths):#[:4]:
        print(filename, i+1, "of", len(paths))
        
        ## FROM: ex_data = sc.read_10x_h5(path)
        f = tables.open_file(str(filename), 'r')
        v3 = '/matrix' in f
        
        ## FROM: adata = sc.readwrite._read_v3_10x_h5(filename)
        dsets = {}
        for node in f.walk_nodes('/matrix', 'Array'):
            dsets[node.name] = node.read()
            #print("%s class: %s" % (node.name, dsets[node.name]))
        ## 7.18s
        M, N = dsets['shape']
        data = dsets['data']
        if dsets['data'].dtype == np.dtype('int32'):
            data = dsets['data'].view('float32')
            data[:] = dsets['data']
        ## 7.4s     
        matrix = csr_matrix(
            #(cp.array(data), cp.array(dsets['indices']), cp.array(dsets['indptr'])), shape=(N, M),
            (data, dsets['indices'], dsets['indptr']), shape=(N, M),
        )        
        ## 8.09s
        genome =  dsets['genome'].astype(str)        
        var_names = dsets['name'].astype(str)
        gene_ids = dsets['id'].astype(str)
        feature_types = dsets['feature_type'].astype(str)
        obs = dsets['barcodes']
        obs_names = obs.astype(str) # slow
        #obs_names = [str(v) for v in obs] # slower :(
        #print("barcodes: %s" % obs)
        #print("dsets: %s" % dsets.keys())
        args = list([
            matrix,
            dict(obs_names=obs_names),
            dict(var_names=var_names,gene_ids=gene_ids,feature_types=feature_types,genome=genome)
        ])
        adata = AnnData(*args)
        # 21.6s


Overwriting load_file.py


In [9]:
import load_file
import importlib
importlib.reload(load_file)
from load_file import load_file


In [10]:
!sudo bash -c 'sync; echo 1 >| /proc/sys/vm/drop_caches'; echo RAM cache cleared

RAM cache cleared


In [11]:
%lprun -T load_file -f load_file load_file(paths)

/mnt/ssd/moredata/CID003192-1.h5ad 1 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003070-1.h5ad 2 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003071-1.h5ad 3 of 34


...

/mnt/ssd/moredata/CID003807-1.h5ad 33 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003806-1.h5ad 34 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.



*** Profile printout saved to text file 'load_file'. 


In [12]:
print(open('load_file', 'r').read())

Timer unit: 1e-06 s

Total time: 23.6868 s
File: /home/ssmith/ctxbio/speedup/load_file.py
Function: load_file at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           def load_file(paths):
    14        35        108.0      3.1      0.0      for i, filename in enumerate(paths):#[:4]:
    15        34      64489.0   1896.7      0.3          print(filename, i+1, "of", len(paths))
    16                                                   
    17                                                   ## FROM: ex_data = sc.read_10x_h5(path)
    18        34      78385.0   2305.4      0.3          f = tables.open_file(str(filename), 'r')
    19        34       7741.0    227.7      0.0          v3 = '/matrix' in f
    20                                                   
    21                                                   ## FROM: adata = sc.readwrite._read_v3_10x_h5(filename)
    22        34       2994.0     88.1      0.0    

In [13]:
%%time
from load_file import load_file
load_file(paths)

/mnt/ssd/moredata/CID003192-1.h5ad 1 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003070-1.h5ad 2 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003071-1.h5ad 3 of 34


...

/mnt/ssd/moredata/CID003807-1.h5ad 33 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003806-1.h5ad 34 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CPU times: user 20 s, sys: 1.75 s, total: 21.7 s
Wall time: 21.7 s


Timings:
```
CPU times: user 20.1 s, sys: 1.57 s, total: 21.7 s
Wall time: 21.6 s
```

In [20]:
%%time
adata_raw = {}
paths = glob.glob(os.path.join(basepath, "*.h5ad"))
for i, path in enumerate(paths):#[:4]:
    print(path, i+1, "of", len(paths))
    name = os.path.basename(path).split(".")[0]
    cur_adata = sc.read_10x_h5(path)
    adata_raw[name] = cur_adata
list(adata_raw.values())

/mnt/ssd/moredata/CID003192-1.h5ad 1 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003070-1.h5ad 2 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003071-1.h5ad 3 of 34


...

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/ssd/moredata/CID003806-1.h5ad 34 of 34


Variable names are not unique. To make them unique, call `.var_names_make_unique`.


CPU times: user 21.2 s, sys: 2.8 s, total: 24 s
Wall time: 24 s


[View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 View of AnnData object with n_obs × n_vars = 737280 × 33538 
     var: 'gene_ids', 'feature_types', 'genome',
 

Timings: 
```
CPU times: user 21.5 s, sys: 1.82 s, total: 23.4 s
Wall time: 23.3 s
```
Reading the files CPU-bound when accesssing `/mnt/ebs_gp2/`, and consumes most of the time.  Switching to `ssd` provides no benefit, as expected.

In [21]:
%%time
paths = glob.glob(os.path.join(basepath, "*.h5ad"))
for i, path in enumerate(paths):#[:4]:
    print(path, i+1, "of", len(paths))
    name = os.path.basename(path).split(".")[0]
    cur_adata = adata_raw[name]
    cur_adata.var_names_make_unique()

/mnt/ssd/moredata/CID003192-1.h5ad 1 of 34
/mnt/ssd/moredata/CID003070-1.h5ad 2 of 34
/mnt/ssd/moredata/CID003071-1.h5ad 3 of 34
/mnt/ssd/moredata/CID003158-1.h5ad 4 of 34
/mnt/ssd/moredata/CID003069-1.h5ad 5 of 34
/mnt/ssd/moredata/CID003072-1.h5ad 6 of 34
/mnt/ssd/moredata/CID003193-1.h5ad 7 of 34
/mnt/ssd/moredata/CID003158-2.h5ad 8 of 34
/mnt/ssd/moredata/CID003185-1.h5ad 9 of 34
/mnt/ssd/moredata/CID003195-1.h5ad 10 of 34
/mnt/ssd/moredata/CID003186-1.h5ad 11 of 34
/mnt/ssd/moredata/CID003188-1.h5ad 12 of 34
/mnt/ssd/moredata/CID003187-1.h5ad 13 of 34
/mnt/ssd/moredata/CID003572-1.h5ad 14 of 34
/mnt/ssd/moredata/CID003194-1.h5ad 15 of 34
/mnt/ssd/moredata/CID003198-1.h5ad 16 of 34
/mnt/ssd/moredata/CID003196-1.h5ad 17 of 34
/mnt/ssd/moredata/CID003412-1.h5ad 18 of 34
/mnt/ssd/moredata/CID003197-1.h5ad 19 of 34
/mnt/ssd/moredata/CID003580-1.h5ad 20 of 34
/mnt/ssd/moredata/CID003199-1.h5ad 21 of 34
/mnt/ssd/moredata/CID003331-1.h5ad 22 of 34
/mnt/ssd/moredata/CID003413-1.h5ad 23 of 

Timings: 
```
CPU times: user 145 ms, sys: 4.42 ms, total: 149 ms
Wall time: 143 ms
```
Making the names unique takes negligible time.

In [22]:
%%time
adata_collection = {}
paths = glob.glob(os.path.join(basepath, "*.h5ad"))
for i, path in enumerate(paths):#[:4]:
    print(path, i+1, "of", len(paths))
    name = os.path.basename(path).split(".")[0]
    cur_adata = adata_raw[name]
    cur_adata = cur_adata[cur_adata.X.sum(axis=1).A.flatten()>250].copy()
    cur_adata.obs["sample"] = name
    print("  ", cur_adata.shape)
    if cur_adata.shape[0] < 500: continue
    adata_collection[name] = cur_adata    

/mnt/ssd/moredata/CID003192-1.h5ad 1 of 34
   (4567, 33538)
/mnt/ssd/moredata/CID003070-1.h5ad 2 of 34
   (6145, 33538)
/mnt/ssd/moredata/CID003071-1.h5ad 3 of 34
   (8936, 33538)
/mnt/ssd/moredata/CID003158-1.h5ad 4 of 34
   (8141, 33538)
/mnt/ssd/moredata/CID003069-1.h5ad 5 of 34
   (8394, 33538)
/mnt/ssd/moredata/CID003072-1.h5ad 6 of 34
   (11078, 33538)
/mnt/ssd/moredata/CID003193-1.h5ad 7 of 34
   (2159, 33538)
/mnt/ssd/moredata/CID003158-2.h5ad 8 of 34
   (24979, 33538)
/mnt/ssd/moredata/CID003185-1.h5ad 9 of 34
   (3196, 33538)
/mnt/ssd/moredata/CID003195-1.h5ad 10 of 34
   (1996, 33538)
/mnt/ssd/moredata/CID003186-1.h5ad 11 of 34
   (2724, 33538)
/mnt/ssd/moredata/CID003188-1.h5ad 12 of 34
   (2665, 33538)
/mnt/ssd/moredata/CID003187-1.h5ad 13 of 34
   (2561, 33538)
/mnt/ssd/moredata/CID003572-1.h5ad 14 of 34
   (6, 33538)
/mnt/ssd/moredata/CID003194-1.h5ad 15 of 34
   (4047, 33538)
/mnt/ssd/moredata/CID003198-1.h5ad 16 of 34
   (2449, 33538)
/mnt/ssd/moredata/CID003196-1.h5ad

Timings: 
```
/mnt/ssd_gp2/moredata/*
CPU times: user 1.03 s, sys: 199 ms, total: 1.23 s
Wall time: 1.22 s
```
The flattening operation takes a small but measurable amount of time.

(in our current pipeline, we actually do a single concatenation of a really big matrix, which is kind of slow but better than the alternative of having to re-do the concatenations which is truly cumbersome with our current infrastructure)

In [23]:
%%time
samples = list(adata_collection.values())
together = samples.pop()
together = together.concatenate(*samples)

CPU times: user 3 s, sys: 384 ms, total: 3.38 s
Wall time: 3.39 s


Timings:
```
/mnt/ssd_gp2/moredata/*
CPU times: user 2.94 s, sys: 360 ms, total: 3.3 s
Wall time: 3.3 s
```
Concatenation takes just over 3s here.  But in real world scenarios this is a barrier.

When the samples list is 5x the size, the time goes up >7x?
```
CPU times: user 21.7 s, sys: 2.31 s, total: 24 s
Wall time: 24 s
```

In [24]:
%%time
together = SCData(together)

together.raw = together
together.uns["species"] = "Human"
together

CPU times: user 51 µs, sys: 7 µs, total: 58 µs
Wall time: 61 µs


AnnData object with n_obs × n_vars = 138091 × 33538 
    obs: 'batch', 'sample'
    var: 'gene_ids', 'feature_types', 'genome-0', 'genome-1', 'genome-2', 'genome-3', 'genome-4', 'genome-5', 'genome-6', 'genome-7', 'genome-8', 'genome-9', 'genome-10', 'genome-11', 'genome-12', 'genome-13', 'genome-14', 'genome-15', 'genome-16', 'genome-17', 'genome-18', 'genome-19', 'genome-20', 'genome-21', 'genome-22', 'genome-23', 'genome-24', 'genome-25'
    uns: 'species'

In [25]:
together.obs["batch"] = together.obs["batch"].astype(int)

I haven't carefully profiled the preprocess/reprocess step below, but I believe about half the time is spent using an inefficient clustering algorithm that we're in the process of upgrading. The other big step is training the deep learning model, and I don't think that'll be something we can do a whole lot to improve.

In [26]:
%%time
reprocessed = SCData(together).reprocess()

[2020-03-09 10:00:13,419] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).


Training scVI on: View of AnnData object with n_obs × n_vars = 138091 × 1372 
    obs: 'batch', 'sample'
    var: 'gene_ids', 'feature_types', 'genome-0', 'genome-1', 'genome-2', 'genome-3', 'genome-4', 'genome-5', 'genome-6', 'genome-7', 'genome-8', 'genome-9', 'genome-10', 'genome-11', 'genome-12', 'genome-13', 'genome-14', 'genome-15', 'genome-16', 'genome-17', 'genome-18', 'genome-19', 'genome-20', 'genome-21', 'genome-22', 'genome-23', 'genome-24', 'genome-25'
    uns: 'species', 'mito_genes'


[2020-03-09 10:00:14,302] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-03-09 10:00:14,309] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-03-09 10:00:14,627] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-03-09 10:00:14,915] INFO - scvi.dataset.dataset | Downsampled from 138091 to 138091 cells


Training LDVAE model.
training:  11%|█         | 110/1000 [10:06<1:21:39,  5.51s/it]

[2020-03-09 10:10:36,991] INFO - scvi.inference.trainer | 
Stopping early: no improvement of more than 2 nats in 100 epochs
[2020-03-09 10:10:36,992] INFO - scvi.inference.trainer | If the early stopping criterion is too strong, please instantiate it with different parameters in the train method.


...

120000 0.8689921863119248


[2020-03-09 10:11:27,946] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2020-03-09 10:11:27,983] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-03-09 10:11:27,984] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-03-09 10:11:28,003] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-03-09 10:11:28,016] INFO - scvi.dataset.dataset | Downsampled from 8091 to 8091 cells


130000 0.9414082018379185


/home/ssmith/benchmark/env/lib/python3.7/site-packages/umap/umap_.py:328: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fuzzy_simplicial_set" failed type inference due to: Untyped global name 'nearest_neighbors': cannot determine Numba type of <class 'function'>

File "../../benchmark/env/lib/python3.7/site-packages/umap/umap_.py", line 446:
def fuzzy_simplicial_set(
    <source elided>
    if knn_indices is None or knn_dists is None:
        knn_indices, knn_dists, _ = nearest_neighbors(
        ^

  @numba.jit()
/home/ssmith/benchmark/env/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "fuzzy_simplicial_set" was compiled in object mode without forceobj=True.

File "../../benchmark/env/lib/python3.7/site-packages/umap/umap_.py", line 329:
@numba.jit()
def fuzzy_simplicial_set(
^

  self.func_ir.loc))


CPU times: user 2h 50min, sys: 7min 34s, total: 2h 57min 35s
Wall time: 41min 37s


In [27]:
reprocessed

AnnData object with n_obs × n_vars = 138091 × 33538 
    obs: 'batch', 'sample', 'clusters', 'louvain'
    var: 'gene_ids', 'feature_types', 'genome-0', 'genome-1', 'genome-2', 'genome-3', 'genome-4', 'genome-5', 'genome-6', 'genome-7', 'genome-8', 'genome-9', 'genome-10', 'genome-11', 'genome-12', 'genome-13', 'genome-14', 'genome-15', 'genome-16', 'genome-17', 'genome-18', 'genome-19', 'genome-20', 'genome-21', 'genome-22', 'genome-23', 'genome-24', 'genome-25'
    uns: 'species', 'mito_genes', 'scVI_imputed_genes', 'scVI_train_history', 'units', 'neighbors', 'leiden', 'louvain'
    obsm: 'scVI_latent', 'scVI_imputed_data', 'X_tsne', 'X_umap'

In [28]:
reprocessed.X

<138091x33538 sparse matrix of type '<class 'numpy.float32'>'
	with 104192386 stored elements in Compressed Sparse Row format>

In [29]:
"density:", reprocessed.X.nnz/(reprocessed.X.shape[0]*reprocessed.X.shape[1])

('density:', 0.022497457935077618)

After processing the data, we typically do some exploratory data analysis to identify cell populations of interest.

We use the `vaex` library to summarize data in a gridded fashion prior to plotting using matplotlib, as matplotlib alone is incapable of rendering large numbers of points.

In [33]:
reprocessed.qplot("PTPRC", vaex=False, pointsize=1, scale=1.5)

In [35]:
_, ax = plt.subplots(figsize=(8,8))
reprocessed.catplot("clusters", vaex=False, ax=ax, facet=False)

One thing we do frequently is zoom in on a subset of cells by retraining our dimensionality reduction functions on only that subset of cells (eg T cells or endothelial cells). Then, we'd like to carry annotations made in those subviews back to the main dataset.

In [36]:
cluster2_adata = reprocessed[reprocessed.obs.query("clusters=='12'").index].reprocess()

[2020-03-09 12:52:42,755] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
/home/ssmith/benchmark/env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ssmith/benchmark/env/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/ssmith/benchmark/env/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3584: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
/home/ssmith/benchmark/env/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/ssmith/benchmark/env/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
[2020-03-09 12:52:42,762] INFO

Training scVI on: View of AnnData object with n_obs × n_vars = 4110 × 319 
    obs: 'batch', 'sample', 'clusters', 'louvain'
    var: 'gene_ids', 'feature_types', 'genome-0', 'genome-1', 'genome-2', 'genome-3', 'genome-4', 'genome-5', 'genome-6', 'genome-7', 'genome-8', 'genome-9', 'genome-10', 'genome-11', 'genome-12', 'genome-13', 'genome-14', 'genome-15', 'genome-16', 'genome-17', 'genome-18', 'genome-19', 'genome-20', 'genome-21', 'genome-22', 'genome-23', 'genome-24', 'genome-25'
    uns: 'species', 'mito_genes'
Training LDVAE model.
training:  28%|██▊       | 278/1000 [00:39<01:43,  6.99it/s]

[2020-03-09 12:53:22,762] INFO - scvi.inference.trainer | 
Stopping early: no improvement of more than 2 nats in 100 epochs
[2020-03-09 12:53:22,763] INFO - scvi.inference.trainer | If the early stopping criterion is too strong, please instantiate it with different parameters in the train method.


training:  28%|██▊       | 279/1000 [00:39<01:43,  6.99it/s]


[2020-03-09 12:53:22,882] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2020-03-09 12:53:22,889] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-03-09 12:53:22,889] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-03-09 12:53:22,892] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-03-09 12:53:22,897] INFO - scvi.dataset.dataset | Downsampled from 4110 to 4110 cells


0 0.0


/home/ssmith/benchmark/env/lib/python3.7/site-packages/umap/umap_.py:328: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fuzzy_simplicial_set" failed type inference due to: Untyped global name 'nearest_neighbors': cannot determine Numba type of <class 'function'>

File "../../benchmark/env/lib/python3.7/site-packages/umap/umap_.py", line 446:
def fuzzy_simplicial_set(
    <source elided>
    if knn_indices is None or knn_dists is None:
        knn_indices, knn_dists, _ = nearest_neighbors(
        ^

  @numba.jit()
/home/ssmith/benchmark/env/lib/python3.7/site-packages/numba/compiler.py:742: NumbaWarning: Function "fuzzy_simplicial_set" was compiled in object mode without forceobj=True.

File "../../benchmark/env/lib/python3.7/site-packages/umap/umap_.py", line 329:
@numba.jit()
def fuzzy_simplicial_set(
^

  self.func_ir.loc))


In [37]:
cluster2_adata.catplot("clusters")

In [38]:
cluster2_adata.diff_exp().head(10)

... storing 'sample' as categorical
... storing 'feature_types' as categorical
... storing 'genome-0' as categorical
... storing 'genome-1' as categorical
... storing 'genome-2' as categorical
... storing 'genome-3' as categorical
... storing 'genome-4' as categorical
... storing 'genome-5' as categorical
... storing 'genome-6' as categorical
... storing 'genome-7' as categorical
... storing 'genome-8' as categorical
... storing 'genome-9' as categorical
... storing 'genome-10' as categorical
... storing 'genome-11' as categorical
... storing 'genome-12' as categorical
... storing 'genome-13' as categorical
... storing 'genome-14' as categorical
... storing 'genome-15' as categorical
... storing 'genome-16' as categorical
... storing 'genome-17' as categorical
... storing 'genome-18' as categorical
... storing 'genome-19' as categorical
... storing 'genome-20' as categorical
... storing 'genome-21' as categorical
... storing 'genome-22' as categorical
... storing 'genome-23' as categor

,0,1,2,3,4,5,6,7,8
0,IGHA1,AP003086.2,INPP4B,USP54,BANK1,AOAH,DHFR,DHFR,AGAP1
1,FNDC3B,USP54,MBNL1,AP003086.2,AFF3,PRKCH,MTRNR2L8,CD96,DOCK1
2,MZB1,CD96,BCL11B,TAOK1,RALGPS2,ITGA1,MTRNR2L12,AOAH,LGALS4
3,IGKC,PGGHG,ANK3,MACROD2,ARHGAP24,ITGAE,MTRNR2L1,ITGAE,ADAMTSL1
4,IGHA2,ZAP70,TC2N,TMEM131,EBF1,CD96,FP236383.1,MTRNR2L8,MAGI3
5,RRBP1,PITPNC1,ARHGAP15,SETD3,MS4A1,SKAP1,FP671120.1,MTRNR2L12,ABCC3
6,IGLL5,PBXIP1,PTPRC,CSMD1,PAX5,MBNL1,MTRNR2L10,PITPNC1,LLGL2
7,CDK14,ATP8B4,RIPOR2,HEPHL1,WDFY4,FYN,AC105402.3,PRKCH,TCF7L2
8,ELL2,GRAMD1B,LEF1,FGF12,FCHSD2,TRG-AS1,KIF5C,ITGA1,SH3D19
9,FNDC3A,CD247,RUNX1,AL050403.2,BCL11A,PITPNC1,ASIP,THEMIS,NR3C2


In [39]:
cluster2_adata.assign_labels("clusters", "cell_type", {
    ("0","1","2"):"Cell Type X",
    ("3","4","5"):"Cell Type Y",
    ("6","7"):"Cell Type Z",
    "8":"W Cells"
})

In [40]:
cluster2_adata.catplot("cell_type")

In [41]:
reprocessed.obs["cell_type"] = "Unknown"
reprocessed.obs.loc[cluster2_adata.obs_names, "cell_type"] = cluster2_adata.obs["cell_type"]

In [42]:
reprocessed.catplot("cell_type", vaex=True)

ImportError: /home/ssmith/benchmark/env/lib/python3.7/site-packages/vaex/strings.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZNSt14regex_iteratorIN9__gnu_cxx17__normal_iteratorIPKcSsEEcSt12regex_traitsIcEEC1Ev